# X Crawler

With help of [this link](https://github.com/umapornp/twitter-crawler).

In [66]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from datetime import date
from datetime import time

In [ ]:
os.mkdir('tweets')
os.chdir('selenium-twitter-scraper')

os.system('python scraper -t 5 --hashtag "فمنیست" --top')
os.system('python scraper -t 5 --query "فقط زن" --top')
os.system('python scraper -t 5 --query "فقط مرد" --top')

os.system('cp -r tweets ../')
os.chdir('..')

In [ ]:
print()